In [9]:
import sys
import os
import time
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
import joblib
from joblib import dump, load
import xgboost as xgb
import matplotlib.animation as animation
from typing import Dict, Any
from copy import deepcopy
import random 
from sklearn.base import BaseEstimator
from torch import nn
import torch
from torch.distributions.categorical import Categorical
from sklearn.utils import gen_batches

# Add the src directory to the path
sys.path.append(os.path.abspath('../src'))
sys.path.append(os.path.abspath('..'))

# Import the BaseAgent class
from src.agents.base_agent import BaseAgent
from initial_windfields import get_initial_windfield, INITIAL_WINDFIELDS
from src.env_sailing import SailingEnv
from src.test_agent_validity import validate_agent, load_agent_class
from src.evaluation import evaluate_agent, visualize_trajectory

# Environment parameters
env = SailingEnv(**get_initial_windfield('simple_static'))
n_actions = env.action_space.n
d_s = 2054

### Define the playing function

In [95]:
def make_animation(imgs):
  """
  Makes an animation from a list of images
  Parameters
  ----------
  imgs: list of (height, width, 3) np arrays
    list of images
  Return
  -------
  ani: animation
  """
  fig, ax = plt.subplots()
  draw = []
  for i in range(len(imgs)):
    draw_i = ax.imshow(imgs[i])
    if i == 0:
      ax.imshow(imgs[0]) # Show an initial one first
    draw.append([draw_i])
  plt.close()
  ani = animation.ArtistAnimation(fig, draw, interval=200, blit=True,
                              repeat=False)
  return ani

In [96]:
def play_policy(env, pi, horizon=200, capture_rate=1):
  s, _ = env.reset()
  a = pi(s)
  imgs = []
  imgs.append(env.render())
  for tt in range(horizon):
    s, rew, term, trunc, _ = env.step(a)
    a = pi(s)
    if tt % capture_rate == 0:
      imgs.append(env.render())
    if term or trunc:
      break
  return make_animation(imgs)

In [97]:
for initial_windfield_name, initial_windfield in INITIAL_WINDFIELDS.items():
    print(initial_windfield_name)
    print(initial_windfield)

training_1
{'wind_init_params': {'base_speed': 3.0, 'base_direction': (-0.8, -0.2), 'pattern_scale': 32, 'pattern_strength': 0.3, 'strength_variation': 0.4, 'noise': 0.1}, 'wind_evol_params': {'wind_change_prob': 1.0, 'pattern_scale': 128, 'perturbation_angle_amplitude': 0.1, 'perturbation_strength_amplitude': 0.1, 'rotation_bias': 0.02, 'bias_strength': 1.0}, 'env_params': {'wind_grid_density': 25, 'wind_arrow_scale': 80, 'render_mode': 'rgb_array'}}
training_2
{'wind_init_params': {'base_speed': 3.0, 'base_direction': (-0.2, 0.8), 'pattern_scale': 128, 'pattern_strength': 0.6, 'strength_variation': 0.3, 'noise': 0.1}, 'wind_evol_params': {'wind_change_prob': 1.0, 'pattern_scale': 128, 'perturbation_angle_amplitude': 0.1, 'perturbation_strength_amplitude': 0.1, 'rotation_bias': 0.02, 'bias_strength': 1.0}, 'env_params': {'wind_grid_density': 25, 'wind_arrow_scale': 80, 'render_mode': 'rgb_array'}}
training_3
{'wind_init_params': {'base_speed': 3.0, 'base_direction': (0.2, -0.8), 'patt

## Fitted Q iteration

>Implement fitted Q iterations with random forest using uniform exploration for $\pi$.

In [5]:
# Collect a dataset 

def collect_dataset(pi= lambda x: np.random.randint(0, n_actions) , n=10000):
    """
    Collect a dataset of the form $(s_i, a_i, r_{a_i}(s_i), s'_i)_{i=1}^n$
    by running a policy that chooses its action uniformly at random

    Parameters
    ---------
    env: Environment
    pi: Policy
    n: int
        Number of samples to collect

    Return
    -----
    data: np array of size (n, 2d_s + d_a + 2)
        data collected by the random policy
        the first 4 columns are the states s_i,
        the 5th, 6th and 7th columns contain the actions a_i,
        rewards r_{a_i}(s_i) and whether the step is a termination step
        and the columns 8th-12th columns contain the states s'_i
    """
    data = []
    for initial_windfield_name, initial_windfield in INITIAL_WINDFIELDS.items():
        env = SailingEnv(**get_initial_windfield(initial_windfield_name))
        s0, _ = env.reset()
        s = s0.copy()
        n_actions = env.action_space.n

        for i in range(n//4):
            a = pi(s)
            s2, r, done, trunc, _ = env.step(a)
            data.append(s.copy().tolist() + [a, r, done] + s2.copy().tolist())
            if done or trunc:
                s, _ = env.reset()
            else:
                s = s2.copy()

    return np.array(data)

In [6]:
class FQI(BaseAgent):
    """ FQI agent"""
    
    def __init__(self, data=collect_dataset()):
        super().__init__()
        self.d_s = 2054
        self.data = data
        self.gamma = 0.99 
        self.n_iterations = 100
        self.n_actions = 9
        self.epsilon = 0.2
        self.model = RandomForestRegressor()
        self.pi = None
        
    
    def act(self, observation: np.ndarray) -> int:
        if self.pi is None:
            print("The Agent has not been trained")
        else:
            return int(self.pi(observation))
    
    def trainFQI(self):
        # Use the data collected before
        n = len(self.data)
        states, actions, rewards, dones, next_states = self.data[:, :2054], self.data[:, 2054], self.data[:, 2054+1], self.data[:, 2054+2], self.data[:, 2054+3:]
        X, Y = self.data[:, :2054+1], self.data[:, 2054+1]
        self.model.fit(X, Y)

        for _ in tqdm(range(self.n_iterations)):
            Qmax = np.max(
                            [
                            self.model.predict(np.column_stack([
                                            self.data[:, d_s + 3:],
                                            np.ones(n).reshape(-1, 1) * a
                                            ]))
                                for a in range(n_actions)
                            ],axis=0)
            Y = self.data[:, d_s + 1] + self.gamma * (1 - dones) * Qmax
            self.model.fit(X, Y)
        
        pi = lambda s: np.argmax(
                                [self.model.predict(np.array(s.tolist() + [a]).reshape(1, -1))[0] for a in range(n_actions)]
                                )
        self.pi = pi
        agent.save(path="models/FQI_RF")
        return pi
    
    def reset(self) -> None:
        """Reset the agent."""
        pass  # Nothing to reset in this simple agent
    
    def seed(self, seed: int = None) -> None:
        """Set the random seed."""
        self.np_random = np.random.default_rng(seed)

    def save(self, path):
        if self.model is not None:
            joblib.dump(self.model, path)
        else:
            print("No model found to save.")

    def load(self):
        try:
            self.model = joblib.load("models/FQI_RF")
        except:
            print("No saved model found.")
            self.model = None


agent = FQI()
pi_FQI = agent.trainFQI()

100%|██████████| 100/100 [33:55:12<00:00, 1221.13s/it]  


In [8]:
# Choose which training initial windfields to evaluate on
TRAINING_INITIAL_WINDFIELDS = ["simple_static", "training_1", "training_2", "training_3"]

# Evaluation parameters for all initial windfields
ALL_SEEDS = [42, 43, 44, 45, 46]  # Seeds to use for all evaluations
ALL_MAX_HORIZON = 200             # Maximum steps per episode

# Only run if the agent was successfully loaded
if 'agent' in locals():
    # Store results for each initial windfield
    all_results = {}
    
    print(f"Evaluating agent on {len(TRAINING_INITIAL_WINDFIELDS)} training initial windfields...")
    
    # Evaluate on each initial windfield
    for initial_windfield_name in TRAINING_INITIAL_WINDFIELDS:
        print(f"\nInitial windfield: {initial_windfield_name}")
        
        # Get the initial windfield
        initial_windfield = get_initial_windfield(initial_windfield_name)
        
        # Run the evaluation
        results = evaluate_agent(
            agent=agent,
            initial_windfield=initial_windfield,
            seeds=ALL_SEEDS,
            max_horizon=ALL_MAX_HORIZON,
            verbose=False,  # Less verbose for multiple evaluations
            render=False,
            full_trajectory=False
        )
        
        # Store results
        all_results[initial_windfield_name] = results
        
        # Print summary
        print(f"  Success Rate: {results['success_rate']:.2%}")
        print(f"  Mean Reward: {results['mean_reward']:.2f}")
        print(f"  Mean Steps: {results['mean_steps']:.1f}")
    
    # Print overall performance
    total_success = sum(r['success_rate'] for r in all_results.values()) / len(all_results)
    print("\n" + "="*50)
    print(f"OVERALL SUCCESS RATE: {total_success:.2%}")
    print("="*50)

Evaluating agent on 4 training initial windfields...

Initial windfield: simple_static
  Success Rate: 0.00%
  Mean Reward: 0.00
  Mean Steps: 200.0

Initial windfield: training_1
  Success Rate: 20.00%
  Mean Reward: 7.70
  Mean Steps: 179.2

Initial windfield: training_2
  Success Rate: 0.00%
  Mean Reward: 0.00
  Mean Steps: 200.0

Initial windfield: training_3
  Success Rate: 20.00%
  Mean Reward: 3.12
  Mean Steps: 197.2

OVERALL SUCCESS RATE: 10.00%


## DQN

In [63]:
class FCNet(torch.nn.Module):
  """
  Define the Neural network with the __init__ and forward method.
  It should define a fully connected
  neural network with prescribed input size, hidden size and output size
  """
  def __init__(self, input_size, hidden_size, output_size) -> None:
    super().__init__()
    self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size)
            )

  def forward(self, x):
    return self.linear_relu_stack(x)


class NN(nn.Module):
  """
  A sklearn-like class for the neural net
  """
  def __init__(self, n_iterations, input_size, hidden_size, output_size, alpha, seed, batch_size) -> None:
    """
    Initialize the sklearn class:
    - Record the input parameters using
    self.parameter = parameter
    - Initialize the neural network model and record it in self.model
    - Initialize the Adam optimizer and record it in self.optimizer
    """
    super().__init__()
    self.device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
    self.n_iterations = n_iterations
    self.alpha = alpha
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.seed = seed
    self.batch_size = batch_size
    self.model = FCNet(input_size, hidden_size, output_size).to(self.device)
    self.optimizer = torch.optim.Adam(self.model.parameters(), lr=alpha)


  def partial_fit(self, X, Y):
    """Update parameters
    - Convert numpy input data to torch
    - Define the loss
    - Find the gradient via automatic differentiation
    - Perform a gradient update of the parameters

    Parameters
    ----------
    X: np array of size (n, ds + da)
      state-action pairs
    Y: np array of size (n, 1)
      Estimate of the state action value function
    """
    # DATA
    Xs = torch.from_numpy(X[:, :-1].copy()).float().to(self.device)  # états
    Xa = torch.from_numpy(X[:, -1].copy()).long().to(self.device)    # actions (indices)
    if isinstance(Y, np.ndarray):
        Y = torch.from_numpy(Y.copy()).float().to(self.device)
    else:
        Y = Y.float().to(self.device)

    # LOSS
    loss_fn = torch.nn.MSELoss()
    Q_all = self.model(Xs)
    Q_pred = Q_all[torch.arange(len(Xs)), Xa]
    loss = loss_fn(Q_pred, Y)
    loss.backward()
    self.optimizer.step()
    self.optimizer.zero_grad()


  def fit(self, X, Y):
    """Applies n_iterations steps of gradient using function grad_step.
    At each iteration, all samples are shuffled and split into batches of size
    `batch_size`.
    The gradient steps are performed on each batch of data sequentially
    """
    idx_samples = np.arange(len(X))
    for _ in range(self.n_iterations):
      np.random.shuffle(idx_samples)
      for batch_slice in gen_batches(len(X), self.batch_size):
        Xb, Yb = X[idx_samples[batch_slice]].copy(), Y[idx_samples[batch_slice]].copy()
        self.partial_fit(Xb, Yb)

  def predict(self, X):
    """Use the fitted parameter to predict q(s, a)
    - Convert input data to Torch
    - Apply the model
    - Convert the output to numpy

    Parameters
    ---------
    X: np array of shape (n, ds + da)
    Return
    ------
    qSA: np array of shape (n,)
      qSA[i] is an estimate of q(s_i, a_i) computed with the pred function
      where s_i and a_i are given by X[i]
    """
    Xs = X[:, :-1]
    Xa = X[:, -1]
    with torch.no_grad():
      Xs = torch.from_numpy(Xs.copy()).float().to(self.device)
      Xa = torch.from_numpy(Xa.copy()).int().to(self.device)
      y = self.model(Xs)[torch.arange(len(Xs)), Xa]
    return y.cpu().numpy()

In [65]:
class DQNAgent(BaseAgent):
    """ DQN Agent """
    
    def __init__(self):
        super().__init__()
        self.device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
        self.d_s = 2054
        self.n_actions = 9
        self.pi = lambda x: np.random.randint(0, self.n_actions)
        self.capacity = 2000
        self.batch_size = 200
        self.eps = 0.5
        self.gamma = 0.99
        self.C = 20
        self.n_iterations = 1000
        self.nb_gradient_steps = 5  # Number of gradient steps after each iteration
        self.model = NN(n_iterations=1,
                        input_size=2054,
                        hidden_size=24,
                        output_size=env.action_space.n,
                        alpha=0.001,
                        batch_size=None,
                        seed=0)
        self.buffer = np.zeros((self.capacity, self.d_s * 2 + 3))
        self.target_model = deepcopy(self.model)
        self.Qmax = None 
        self.memory = []
        self.criterion = torch.nn.SmoothL1Loss()
        self.reset()

    def sample(self):
        batch = random.sample(self.buffer, self.batch_size)
        return list(map(lambda x:torch.Tensor(np.array(x)).to(self.device), list(zip(*batch))))

    def gradient_step(self, t):
        if len(self.buffer) > self.batch_size:
            I = np.random.randint(min(t+1, self.capacity), size=self.batch_size)
            data = self.buffer[I]

            # update target network if needed
            if t % self.C == 0:
                self.target_model = deepcopy(self.model)

            Xb = data[:, :len(self.s) + 1]
            Yb = data[:, len(self.s) + 1]

            # Qmax update
            if self.Qmax is None:
                self.model.partial_fit(Xb, Yb)

            self.Qmax = np.max(
                            [self.target_model.predict(np.column_stack([data[:, len(self.s) + 3:], np.ones(len(data)).reshape(-1, 1) * a])) for a in range(n_actions)],
                            axis=0)
            Yb = data[:, len(self.s) + 1] + self.gamma * (1 - data[:, len(self.s) + 2]) * self.Qmax
            Yb = torch.from_numpy(Yb.copy()).to(torch.float).to(self.model.device)
            self.model.partial_fit(Xb, Yb)

    def greedy_action(self, state):
        return np.random.randint(0, self.n_actions)
    
    def definePI(self):
        def pi(s):
            q = [self.model.predict(np.array(self.s.copy().tolist() + [a]).reshape(1,-1))[0] for a in range(self.n_actions)]
            return np.argmax(q)
        return lambda s: pi(s)

    def trainDQN(self, path):
        for initial_windfield_name, initial_windfield in INITIAL_WINDFIELDS.items():
            env = SailingEnv(**get_initial_windfield(initial_windfield_name))
            state, _ = env.reset()
            epsilon = self.eps
            self.s = state.copy()

            for t in tqdm(range(self.n_iterations)):
                # select epsilon-greedy action
                if np.random.rand() < epsilon:
                    action = env.action_space.sample()
                else:
                    action = self.greedy_action(state)
                # step
                next_state, reward, done, trunc, _ = env.step(action)
                self.buffer[t % self.capacity, :] = np.array(state.copy().tolist() + [action, reward, done] + next_state.copy().tolist()) # [s, a, r, term, s2]

                # train
                for _ in range(self.nb_gradient_steps):   
                    self.gradient_step(t)

                if done or trunc:
                    state, _ = env.reset()
                    self.s = state.copy()
                else:
                    state = next_state
                    self.s = next_state.copy()
        self.save(path)
    
    def act(self, observation: np.ndarray) -> int:
        """ """
        with torch.no_grad():
            Q = self.model(torch.Tensor(observation).unsqueeze(0).to(self.device))
            probs = Categorical(Q)
            action = probs.sample()
        return action
    
    def reset(self) -> None:
        """Reset the agent."""
        self.s0, _ = env.reset()
        self.s = self.s0.copy()
    
    def save(self, path):
        if self.model is not None:
            torch.save(self.model.state_dict(), path)
        else:
            print("No model found to save.")

    def load(self):
        try:
            self.model = NN(n_iterations=1,
                        input_size=2054,
                        hidden_size=24,
                        output_size=env.action_space.n,
                        alpha=0.001,
                        batch_size=None,
                        seed=0).to(self.device)
            self.model.load_state_dict(torch.load("models/DQN", map_location = self.device))
            self.model.eval()
        except:
            print("No saved model found.")
            self.model = None
    
    def seed(self, seed: int = None) -> None:
        """Set the random seed."""
        self.np_random = np.random.default_rng(seed)

agentDQN = DQNAgent()
agentDQN.trainDQN(path="models/DQN")

  0%|          | 0/1000 [00:00<?, ?it/s]

 43%|████▎     | 426/1000 [00:38<00:49, 11.61it/s]

In [52]:
agentDQN.PI(env.reset()[0])  #example

np.int64(2)

In [57]:
!cd '/Users/augustincablant/Documents/GitHub/RL_project_sailing'

In [61]:
!python /Users/augustincablant/Documents/GitHub/RL_project_sailing/src/test_agent_validity.py RL_project_sailing/notebooks/agents/DQN.py

Error: File 'RL_project_sailing/notebooks/agents/DQN.py' not found.


In [62]:
# Choose which training initial windfields to evaluate on
TRAINING_INITIAL_WINDFIELDS = ["simple_static", "training_1", "training_2", "training_3"]

# Evaluation parameters for all initial windfields
ALL_SEEDS = [42, 43, 44, 45, 46]  # Seeds to use for all evaluations
ALL_MAX_HORIZON = 200             # Maximum steps per episode

# Only run if the agent was successfully loaded
if 'agent' in locals():
    # Store results for each initial windfield
    all_results = {}
    
    print(f"Evaluating agent on {len(TRAINING_INITIAL_WINDFIELDS)} training initial windfields...")
    
    # Evaluate on each initial windfield
    for initial_windfield_name in TRAINING_INITIAL_WINDFIELDS:
        print(f"\nInitial windfield: {initial_windfield_name}")
        
        # Get the initial windfield
        initial_windfield = get_initial_windfield(initial_windfield_name)
        
        # Run the evaluation
        results = evaluate_agent(
            agent=agentDQN,
            initial_windfield=initial_windfield,
            seeds=ALL_SEEDS,
            max_horizon=ALL_MAX_HORIZON,
            verbose=False,  # Less verbose for multiple evaluations
            render=False,
            full_trajectory=False
        )
        
        # Store results
        all_results[initial_windfield_name] = results
        
        # Print summary
        print(f"  Success Rate: {results['success_rate']:.2%}")
        print(f"  Mean Reward: {results['mean_reward']:.2f}")
        print(f"  Mean Steps: {results['mean_steps']:.1f}")
    
    # Print overall performance
    total_success = sum(r['success_rate'] for r in all_results.values()) / len(all_results)
    print("\n" + "="*50)
    print(f"OVERALL SUCCESS RATE: {total_success:.2%}")
    print("="*50)

Evaluating agent on 4 training initial windfields...

Initial windfield: simple_static


TypeError: 'NN' object is not callable